# 5. Unwrap
As we mentioned earlier, for bombardment simulation, we need to change from bulk (all periodic boundaries) to 2D slab (one direction is non-periodic).

## 5.1. Copy files

In [1]:
from pathlib import Path
from shutil import copy
cwd = Path.cwd()

source = cwd.parent/'4. NPT'
copy(source/'ffield.reax.cho', cwd)
copy(source/'data.new.lmp', cwd/'data.lmp')

WindowsPath('c:/Users/fitbe/Desktop/AO Tutorial/LAMMPS/5. Unwrap/data.lmp')

## 5.2. Extract bonds
Extract bonds infomation using `fix reaxff/bonds`

In [3]:
%%writefile in.spe
#-------------------- Initilization ---------------#
units real
boundary p p p
atom_style charge

#-------------------- Structure -------------------#
read_data data.lmp

#-------------------- Force Field -----------------#
pair_style reaxff NULL checkqeq no
pair_coeff * * ffield.reax.cho C H

#-------------------- Simulation ------------------#
# Extract species out
fix bonds all reaxff/bonds 1 bonds.spe
run 0

Overwriting in.spe


In [4]:
!lmp -i in.spe -l log.spe

LAMMPS (2 Aug 2023 - Update 1)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Loaded 1 plugins from C:\Users\fitbe\AppData\Local\LAMMPS 64-bit 2Aug2023 with GUI\plugins
Reading data file ...
  orthogonal box = (0.29115105 0.29115105 0.29115105) to (28.338849 28.338849 28.338849)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  1860 atoms
  reading velocities ...
  1860 velocities
  read_data CPU = 0.044 seconds
Reading potential file ffield.reax.cho with DATE: 2011-02-18

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Your simulation uses code contributions which should be cited:
- pair reaxff command: doi:10.1016/j.parco.2011.08.005
The log file lists these citations in BibTeX format.

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Neighbor list info ...
  update: every = 1 steps, delay = 0 steps, check = yes
  max neighbors/atom: 2000, page size: 100000
  master list

Store bonds information as a dictionary

In [5]:
flag = False
bonds = {}
i = 0

with open('bonds.spe') as f:
    for line in f:
        if 'Number of particles' in line:
            count = int(line.split()[4])
        if flag:
            i += 1
            List = line.split()
            id = int(List[0])
            n = int(List[2])
            nb = list(map(int, List[3:3+n]))
            bonds[id] = nb
            if i == count:
                break
        if '# id type' in line:
            flag = True

print(bonds)

{1315: [1311], 8: [2], 2: [8, 7, 6, 1], 7: [2], 1311: [1315, 1314, 1316, 1308], 1314: [1311, 1319, 1317, 1318], 1322: [1317], 1317: [1314, 1322, 1320, 1321], 6: [2, 10, 11, 9], 10: [6], 1321: [1317], 1197: [1193], 1203: [1199], 1199: [1203, 1204, 1202, 1196], 14: [9], 11: [6], 13: [9], 9: [6, 14, 13, 12], 1: [2, 5, 4, 3], 5: [1], 4: [1], 3: [1], 1193: [1197, 1198, 1196, 1190], 1198: [1193], 1316: [1311], 1204: [1199], 1202: [1199, 1207, 1205, 1206], 1196: [1199, 1193, 1201, 1200], 1201: [1196], 1195: [1190], 1324: [1320], 1328: [1323], 16: [12], 15: [19, 12, 18, 20], 19: [15], 23: [18], 12: [9, 16, 15, 17], 17: [12], 18: [15, 23, 21, 22], 25: [21], 21: [18, 25, 24, 26], 20: [15], 22: [18], 24: [21, 29, 28, 27], 29: [24], 28: [24], 826: [821], 26: [21], 31: [27], 27: [24, 31, 32, 30], 821: [826, 825, 818, 824], 1207: [1202], 822: [818], 818: [821, 822, 815, 823], 828: [824], 824: [821, 828, 829, 827], 34: [30], 832: [827], 32: [27], 1798: [1793], 30: [27, 34, 35, 33], 186: [181], 184: [

Find molecules based on bonds

In [6]:
def find_molecules(bonds):
    molecules = []
    remaining_atoms = set(bonds.keys())
    molecule_ids = {}  # A dictionary to store atom-to-molecule mapping
    molecule_id = 1

    while remaining_atoms:
        start_atom = remaining_atoms.pop()
        molecule = mol_builder(start_atom, [], bonds, molecule_id, molecule_ids)
        molecules.append(molecule)
        remaining_atoms -= set(molecule)
        molecule_id += 1

    return molecules, molecule_ids

def mol_builder(atom, mol, bonds, molecule_id, molecule_ids):
    if atom in mol:
        return mol
    mol.append(atom)
    molecule_ids[atom] = molecule_id  # Assign the molecule ID to the atom
    for i in bonds[atom]:
        mol_builder(i, mol, bonds, molecule_id, molecule_ids)
    return mol

molecules, molecule_ids = find_molecules(bonds)
print("Molecules:", molecules)
print("Molecule IDs:", molecule_ids)

Molecules: [[1, 2, 8, 7, 6, 10, 11, 9, 14, 13, 12, 16, 15, 19, 18, 23, 21, 25, 24, 29, 28, 27, 31, 32, 30, 34, 35, 33, 38, 37, 36, 40, 41, 39, 44, 43, 42, 45, 49, 50, 48, 51, 56, 55, 54, 58, 57, 62, 60, 61, 59, 53, 52, 46, 47, 26, 22, 20, 17, 5, 4, 3], [63, 66, 65, 64, 68, 72, 73, 71, 75, 74, 78, 77, 82, 80, 84, 83, 86, 90, 91, 89, 94, 93, 92, 96, 97, 95, 98, 101, 106, 105, 104, 109, 108, 107, 112, 110, 113, 118, 116, 119, 124, 122, 123, 120, 121, 117, 114, 115, 111, 103, 102, 100, 99, 88, 87, 85, 81, 79, 76, 70, 69, 67], [125, 127, 129, 126, 132, 131, 130, 133, 137, 138, 136, 141, 139, 142, 147, 145, 149, 148, 153, 151, 155, 156, 154, 157, 160, 163, 166, 169, 174, 173, 172, 176, 175, 179, 178, 181, 186, 184, 185, 182, 183, 180, 177, 170, 171, 168, 167, 164, 165, 161, 162, 159, 158, 152, 150, 146, 143, 144, 140, 134, 135, 128], [187, 189, 191, 188, 192, 196, 195, 200, 198, 202, 201, 204, 207, 210, 213, 216, 220, 221, 219, 222, 225, 228, 231, 235, 234, 237, 241, 242, 240, 245, 243, 246,

Unwrap atoms so that all molecules are still connected after converting to non-periodic boundary in z-direction

In [7]:
import ovito.io as io
import numpy as np

# Read 'data.lmp'
pipeline = io.import_file(location='data.lmp', input_format='lammps/data', atom_style='charge')
data = pipeline.compute()
count = data.particles.count
zlo, lz = data.cell[2, 3], data.cell[2, 2]
z = data.particles_['Position_'][:, 2]
molID = molecule_ids
id = data.particles['Particle Identifier']
iz = data.particles['Periodic Image'][:, 2]

# Store atoms in each molecule that have nonzero image flags of z coordinate
unwrap = {}
index = np.nonzero(iz)[0]
num_mol = len(molecules)
for j in range(num_mol):
    unwrap[j+1] = []
for i in index:
    for j in range(num_mol):
        if molID[id[i]] == j + 1:
            unwrap[j+1].append(i)

# Unwrap z coordinate if the atom number is less than original number for each molecule
# In other words, only incomplete molecules are considered
# This function uses the minimum number of atoms in all molecules for determination, which works if the molecules are all the same and do not change after equilibration.
num_molatom = count
for mol in molecules:
    num_molatom = min(num_molatom, len(mol))

for key in unwrap.keys():
    if len(unwrap[key]) < num_molatom:
        for i in unwrap[key]:
            z[i] = z[i] + lz * iz[i]

data.particles_['Position_'][:, 2] = z

io.export_file(data=data, file='data.new.lmp', format='lammps/data', atom_style='charge')

Twice the simulation box in z direction to include unwrapped atoms

In [8]:
with open(f'data.new.lmp', 'r') as f:
    lines = f.readlines()

with open(f'data.new.lmp', 'w') as f:
    for line in lines:
        if 'zlo zhi' in line:
            line = f'{zlo-lz:.2f} {zlo+2*lz:.2f} zlo zhi\n'
        f.write(line)

Open 'data.new.lmp' in Ovito to check if unwrapping is correct 

# 5.3. Species

In [1]:
%%writefile in.spe
#---------- Initization ----------#
units real
dimension 3
boundary p p s          # non-periodic in z-direction
atom_style charge

#---------- structure ----------#
read_data data.new.lmp

#---------- Force Field ----------#
pair_style reaxff NULL checkqeq no
pair_coeff * * ffield.reax.cho C H

#---------- Simulation ----------#
fix 1 all reaxff/species 1 1 1 spe.spe element C H

run 1
#---------- End ----------#

Overwriting in.spe


In [2]:
!lmp -i in.spe -l log.spe

LAMMPS (2 Aug 2023 - Update 3)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Loaded 1 plugins from C:\Users\fitbe\AppData\Local\LAMMPS 64-bit 2Aug2023 with GUI\plugins
Reading data file ...
  orthogonal box = (0.29115105 0.29115105 -27.76) to (28.338849 28.338849 56.39)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  1860 atoms
  reading velocities ...
  1860 velocities
  read_data CPU = 0.020 seconds
Reading potential file ffield.reax.cho with DATE: 2011-02-18

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Your simulation uses code contributions which should be cited:
- pair reaxff command: doi:10.1016/j.parco.2011.08.005
The log file lists these citations in BibTeX format.

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Neighbor list info ...
  update: every = 1 steps, delay = 0 steps, check = yes
  max neighbors/atom: 2000, page size: 100000
  master list distanc

In [3]:
!type spe.spe

#  Timestep    No_Moles    No_Specs      C20H42
          1          30           1          30
